In [1]:
import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
import numpy as np
import json

In [2]:
# CONFIGURANDO A EXIBIÇAO DE REGISTROS NA TELA #

pd.options.display.max_rows = 999
pd.options.display.max_columns=999
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [3]:
arquivo_Alex_NE = 'C:\\Temp\\Frete_Rodrigo\\INDICADORES_FRETE_2021_COMPLETO_zerados.xlsx'
arquivoDatabricks = 'C:\\Temp\\Frete_Rodrigo\\base_databricks.csv'

# IMPORTA ARQUIVO GERADO PELO ALEX PAIVA - ABA "ZERADOS"

In [4]:


dfAZ = pd.read_excel(arquivo_Alex_NE
                      , sheet_name='Zerados'
                      , usecols=[0,1,2]
                      , names=['CodDocVenda'
                               ,'VlrLiqItemOrdem'
                               ,'VlrPesoLiqItem'
                               ]
                      , dtype={'CodDocVenda':'str'
                               ,'VlrLiqItemOrdem':'float'
                               ,'VlrPesoLiqItem':'float'
                               })

dfAZ = dfAZ.drop_duplicates(
  subset = ['CodDocVenda'],
  keep = 'last').reset_index(drop = True)

dfAZ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722 entries, 0 to 721
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CodDocVenda      722 non-null    object 
 1   VlrLiqItemOrdem  722 non-null    float64
 2   VlrPesoLiqItem   722 non-null    float64
dtypes: float64(2), object(1)
memory usage: 17.0+ KB


# IMPORTA ARQUIVO GERADO PELO NOTEBOOK DBS_indicadores_frete_ZLIQ_final - DIA 14/12/2022

In [5]:


dfADB = pd.read_csv(arquivoDatabricks
                    , sep=','
                    , usecols=[0,10,39,40]
                    , names=['AnoMes','CodDocVenda','VlrLiqItemOrdem','VlrPesoLiqItem']
                    , dtype={'AnoMes':'str'
                             ,'CodDocVenda':'str'
                             ,'VlrLiqItemOrdem':'float'
                             ,'VlrPesoLiqItem':'float'
                              }
                    , header=0)

dfADB = dfADB.groupby(['AnoMes','CodDocVenda'], as_index=False).agg(VlrPesoLiqItem=('VlrPesoLiqItem','sum')
                                                   ,VlrLiqItemOrdem=('VlrLiqItemOrdem','sum'))

dfADB.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666231 entries, 0 to 666230
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   AnoMes           666231 non-null  object 
 1   CodDocVenda      666231 non-null  object 
 2   VlrPesoLiqItem   666231 non-null  float64
 3   VlrLiqItemOrdem  666231 non-null  float64
dtypes: float64(2), object(2)
memory usage: 20.3+ MB


### COMPARAÇÃO DA PLANILHA DO ALEX PAIVA ABA - ZERADO (DATAFRAME dfAZ)

### COM O ARQUIVO GERADO PELO NOTEBOOK DBS_indicadores_frete_ZLIQ_final - DIA 14/12/2022 (dfADB)

### O RESULTADO DESSA COMPARAÇÃO É EXPORTADO PARA O ARQUIVO - C:\Temp\Frete_Rodrigo\comparacao_ZERADO.csv

In [87]:
result_ZERADO = pd.merge(left=dfAZ
                              , right=dfADB
                              , left_on='CodDocVenda'
                              , right_on='CodDocVenda'  
                              , how='left')[['AnoMes','CodDocVenda','VlrLiqItemOrdem_x','VlrPesoLiqItem_x']]

result_ZERADO.fillna(-1,inplace=True)

result_ZERADO.rename(columns = {'VlrLiqItemOrdem_x':'VlrLiqItemOrdem_result_ZERADO'
                               ,'VlrPesoLiqItem_x':'VlrPesoLiqItem_result_ZERADO'}, inplace = True)

#result_ZERADO.groupby(['CodDocVenda']).agg(Qtd=('CodDocVenda','count')).sort_values(by='Qtd', ascending=False)

result_ZERADO_Final = result_ZERADO[['CodDocVenda'
               ,'VlrLiqItemOrdem_result_ZERADO'
               ,'VlrPesoLiqItem_result_ZERADO']].drop_duplicates(
  subset = ['CodDocVenda'],
  keep = 'last').reset_index(drop = True)

result_ZERADO_Final.to_csv('C:\\Temp\\Frete_Rodrigo\\comparacao_ZERADO.csv',sep=';', index=False)


### COMPARACAÇÃO DE VALORES DA PLANILHA DO ALEX PAIVA ABA "ZERADOS"

### COM OS RESULT SET DA COMPARAÇÃO DO ARQUIVO DO DATABRICKS VERSOS A PLANILHA DO ALEX PAIVA ABA "ZERADOS"

### RESULTADO GERADO NO ARQUIVO "C:\Temp\Frete_Rodrigo\comparacao_GERAL_ZERADO.csv"

In [90]:
result_GERAL_ZERADO = pd.merge(left=dfAZ
                              , right=result_ZERADO_Final
                              , left_on='CodDocVenda'
                              , right_on='CodDocVenda'  
                              , how='left')[['CodDocVenda'
                                             ,'VlrLiqItemOrdem'
                                             ,'VlrLiqItemOrdem_result_ZERADO'
                                             ,'VlrPesoLiqItem'                                            
                                             ,'VlrPesoLiqItem_result_ZERADO']]


result_GERAL_ZERADO.loc[(result_GERAL_ZERADO['VlrPesoLiqItem']==result_GERAL_ZERADO['VlrPesoLiqItem_result_ZERADO']) & 
                    (result_GERAL_ZERADO['VlrLiqItemOrdem']==result_GERAL_ZERADO['VlrLiqItemOrdem_result_ZERADO']),'Comentario'] = 'Valores IGUAIS de VlrLiqItemOrdem e VlrPesoLiqItem'


result_GERAL_ZERADO.loc[(result_GERAL_ZERADO['VlrPesoLiqItem']==result_GERAL_ZERADO['VlrPesoLiqItem_result_ZERADO']) & 
                    (result_GERAL_ZERADO['VlrLiqItemOrdem']!=result_GERAL_ZERADO['VlrLiqItemOrdem_result_ZERADO']),'Comentario'] = 'Valor de VlrLiqItemOrdem IGUAL e Valor de VlrPesoLiqItem DIFERENTE'


result_GERAL_ZERADO.loc[(result_GERAL_ZERADO['VlrPesoLiqItem']!=result_GERAL_ZERADO['VlrPesoLiqItem_result_ZERADO']) & 
                    (result_GERAL_ZERADO['VlrLiqItemOrdem']==result_GERAL_ZERADO['VlrLiqItemOrdem_result_ZERADO']),'Comentario'] = 'Valor de VlrLiqItemOrdem DIFERENTE e Valor de VlrPesoLiqItem IGUAL'


result_GERAL_ZERADO.loc[(result_GERAL_ZERADO['VlrPesoLiqItem']!=result_GERAL_ZERADO['VlrPesoLiqItem_result_ZERADO']) & 
                    (result_GERAL_ZERADO['VlrLiqItemOrdem']!=result_GERAL_ZERADO['VlrLiqItemOrdem_result_ZERADO']),'Comentario'] = 'Valor de VlrLiqItemOrdem e Valor de VlrPesoLiqItem DIFERENTE'

result_GERAL_ZERADO.loc[((result_GERAL_ZERADO['VlrPesoLiqItem']!=-1) 
                     & (result_GERAL_ZERADO['VlrPesoLiqItem_result_ZERADO']==-1)) & 
                    ((result_GERAL_ZERADO['VlrLiqItemOrdem']!=-1) 
                     & (result_GERAL_ZERADO['VlrLiqItemOrdem_result_ZERADO']==-1)),'Comentario'] = 'Registro Não Encontrado'


result_GERAL_ZERADO.info()
result_GERAL_ZERADO.drop_duplicates(
  subset = ['CodDocVenda'],
  keep = 'last').reset_index(drop = True).to_csv('C:\\Temp\\Frete_Rodrigo\\comparacao_GERAL_ZERADO.csv',sep=';', index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 722 entries, 0 to 721
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   CodDocVenda                    722 non-null    object 
 1   VlrLiqItemOrdem                722 non-null    float64
 2   VlrLiqItemOrdem_result_ZERADO  722 non-null    float64
 3   VlrPesoLiqItem                 722 non-null    float64
 4   VlrPesoLiqItem_result_ZERADO   722 non-null    float64
 5   Comentario                     722 non-null    object 
dtypes: float64(4), object(2)
memory usage: 39.5+ KB


# IMPORTA ARQUIVO GERADO PELO ALEX PAIVA - ABA "NÃO ENCONTRADOS"

In [8]:

dfANE = pd.read_excel(arquivo_Alex_NE
                      , sheet_name='Não encontrados'
                      , usecols=[0]
                      , names=['CodDocVenda_NaoEncontrado']
                      , dtype={'CodDocVenda_NaoEncontrado':'str'})

dfANE = dfANE.drop_duplicates(
  subset = ['CodDocVenda_NaoEncontrado'],
  keep = 'last').reset_index(drop = True)

dfANE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62107 entries, 0 to 62106
Data columns (total 1 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   CodDocVenda_NaoEncontrado  62107 non-null  object
dtypes: object(1)
memory usage: 485.3+ KB


### COMPARAÇÃO DA PLANILHA DO ALEX PAIVA ABA - "NÃO ENCOTRADOS" (DATAFRAME dfANE)

### COM O ARQUIVO GERADO PELO NOTEBOOK DBS_indicadores_frete_ZLIQ_final - DIA 14/12/2022 (dfADB)

### O RESULTADO DESSA COMPARAÇÃO É EXPORTADO PARA O ARQUIVO - C:\Temp\Frete_Rodrigo\comparacao_NAOENCONTRADA.csv

### A BASE "comparacao_NAOENCONTRADA.csv" REPRESENTA OS REGISTROS QUE NÃO FORAM ENCONTRADOS APÓS A COMPARAÇÃO COM A CARGA DO DATABRICKS DO DIA 14/12/2022

In [75]:
result_NE = pd.merge(left=dfANE
                              , right=dfADB
                              , left_on='CodDocVenda_NaoEncontrado'
                              , right_on='CodDocVenda'  
                              , how='left')

result_NE.fillna(-1,inplace=True)

result_NE.groupby(['CodDocVenda']).agg(Qtd=('CodDocVenda','count')).sort_values(by='Qtd', ascending=False)

result_NE.loc[result_NE['CodDocVenda_NaoEncontrado']==result_NE['CodDocVenda'],'Comparacao'] = 'Documento Encontrado'
result_NE.loc[result_NE['CodDocVenda_NaoEncontrado']!=result_NE['CodDocVenda'],'Comparacao'] = 'Documento Não Encontrado'

#result_NE.groupby('Comparacao').agg(Qtd=('Comparacao','count'))

result_GERAL_NE = result_NE[result_NE['Comparacao']=='Documento Não Encontrado']

result_NE[result_NE['Comparacao']=='Documento Não Encontrado'][['CodDocVenda_NaoEncontrado'
                                                                ,'Comparacao']].to_csv('C:\\Temp\\Frete_Rodrigo\\comparacao_NAOENCONTRADA.csv'
                                                                                        ,sep=';'
                                                                                        , index=False
                                                                                         , encoding='latin-1')



# GERA BASE COMPLETA

In [ ]:
result_NE.drop_duplicates(
  subset = ['CodDocVenda_NaoEncontrado'],
  keep = 'last').reset_index(drop = True).to_csv('C:\\Temp\\Frete_Rodrigo\\comparacao_GERAL.csv'
                ,sep=';'
                , index=False
                , encoding='latin-1')


# ABERTURA POR ANO DOS DADOS ENCONTRADOS

In [100]:
analise = result_NE.drop_duplicates(
  subset = ['CodDocVenda_NaoEncontrado'],
  keep = 'last').reset_index(drop = True)

analise['Ano'] = analise['AnoMes'].str[:4]

analise.groupby(['Ano'], as_index=False).agg(Qtd=('Ano','count')).sort_values(by='Ano')

,Ano,Qtd
0,2018,1
1,2020,18
2,2021,60394
3,2022,88


### A BASE "C:\Temp\Frete_Rodrigo\lancamento_contabil.csv" É FRUTO DA EXTRAÇÃO DOS DADOS DA BASE ONPREMISE SGBODS
### RELACIONANDO OS ORDENS DE VENDA NÃO ENCONTRADAS NO ARQUIVO "comparacao_NAOENCONTRADA.csv" 

In [18]:
arquivo_lancamento_contabil = 'C:\\Temp\\Frete_Rodrigo\\lancamento_contabil.csv'

dflancamento_contabil = pd.read_csv(arquivo_lancamento_contabil
                                    , sep=','
                                    ,names=['CodDocVenda','CodDocCompra']
                                    ,dtype={'CodDocVenda':'str','CodDocCompra':'str'}
                                   ,header=0)


dflancamento_contabil = dflancamento_contabil[['CodDocVenda','CodDocCompra']].drop_duplicates(
  subset = ['CodDocVenda','CodDocCompra'],
  keep = 'last').reset_index(drop = True)

dflancamento_contabil

#dflancamento_contabil['CodDocCompra'].drop_duplicates().to_csv('C:\\Temp\\Frete_Rodrigo\\documentos.csv', index=False)

,CodDocVenda,CodDocCompra
0,4001970053,4501818139
1,4001995505,4501832831
2,4002147799,4501994644
3,4002030210,4501867516
4,4001748689,4501617593
...,...,...
1719,4001816816,4501653445
1720,4001734635,4501592292
1721,4001839090,4501695572
1722,4002024209,4501863244


In [19]:
arquivo_lancamento_contabil_ods = 'C:\\Temp\\Frete_Rodrigo\\lancamento_contabil_ods.csv'

dflancamento_contabil_ods = pd.read_csv(arquivo_lancamento_contabil_ods
                                    , sep=';'
                                    ,names=['NumDocCompra','CodTipoRegistro','CodLedger','NomOperacaoRef','Id_Ordem','Id_ElementoPEP','CodElementoPEP']
                                    ,dtype={'NumDocCompra':'str'
                                            ,'CodTipoRegistro':'str'
                                            ,'CodLedger':'str'
                                            ,'NomOperacaoRef':'str'
                                            ,'Id_Ordem':'str'
                                            ,'Id_ElementoPEP':'str'
                                            ,'CodElementoPEP':'str'}
                                   ,header=0)


dflancamento_contabil_ods

#dflancamento_contabil['CodDocCompra'].drop_duplicates().to_csv('C:\\Temp\\Frete_Rodrigo\\documentos.csv', index=False)

,NumDocCompra,CodTipoRegistro,CodLedger,NomOperacaoRef,Id_Ordem,Id_ElementoPEP,CodElementoPEP
0,4501792548,0,2L,RMRP,NaN,63397,CY-2021-RQ-E-07-1073
1,4501672294,0,2L,RMRP,NaN,58793,CY-2021-RQ-E-03-1043
2,4501667930,0,2L,RMRP,NaN,58788,CY-2021-RQ-E-03-1019
3,4501807048,0,2L,RMRP,NaN,58802,CY-2021-RQ-E-03-1081
4,4501972783,0,2L,RMRP,NaN,67670,CY-2021-RQ-E-11-1085
...,...,...,...,...,...,...,...
1546,4501707934,0,2L,RMRP,NaN,59970,CY-2021-RQ-E-04-1073
1547,4501675573,0,2L,RMRP,NaN,56852,CY-2021-RQ-E-01-1027
1548,4501676609,0,2L,RMRP,NaN,58790,CY-2021-RQ-E-03-1031
1549,4501949616,0,2L,RMRP,NaN,67668,CY-2021-RQ-E-11-1081


In [83]:
resultdflancamento_contabil0 = pd.merge(
                                (pd.merge(left=result_GERAL_NE
                              , right=dflancamento_contabil
                              , left_on='CodDocVenda_NaoEncontrado'
                              , right_on='CodDocVenda'  
                              , how='left')
                                ), right=dflancamento_contabil_ods
                              , left_on='CodDocCompra'
                              , right_on='NumDocCompra'
)[['CodDocVenda_y','CodDocVenda_NaoEncontrado','CodDocCompra','Comparacao','CodElementoPEP']]



#resultdflancamento_contabil = pd.merge(
#                                (pd.merge(left=result_GERAL_NE
#                              , right=dflancamento_contabil
#                              , left_on='CodDocVenda_NaoEncontrado'
#                              , right_on='CodDocVenda'  
#                              , how='left')
#                                ), right=dflancamento_contabil_ods
#                              , left_on='CodDocCompra'
#                              , right_on='NumDocCompra'
#    ,how='left'
#)[['CodDocVenda_y','CodDocVenda_NaoEncontrado','CodDocCompra','Comparacao','CodElementoPEP']]


resultdflancamento_contabil0.loc[resultdflancamento_contabil0['CodDocVenda_y']== \
                                resultdflancamento_contabil0['CodDocVenda_NaoEncontrado']
                                ,'Comparacao2'] = 'Registro Encontrado Com ElementoPEP Preencido: ' + resultdflancamento_contabil0['CodElementoPEP']

resultdflancamento_contabil0['Comparacao2'].fillna('NE',inplace=True)

resultdflancamento_contabil0.rename(columns = {'CodDocVenda_y':'CodDocVenda'},inplace=True)

resultdflancamento_contabil0[['CodDocVenda','CodDocCompra','Comparacao2']].drop_duplicates(
  subset = ['CodDocVenda'],
  keep = 'last').reset_index(drop = True).to_csv('C:\\Temp\\Frete_Rodrigo\\comparacao_GERAL_NAOENCONTRADO.csv'
                                   , index=False
                                   , encoding='latin-1'
                                   , sep=';')



### APÓS A GERAÇÃO DO ARQUIVO 'C:\Temp\Frete_Rodrigo\comparacao_GERAL_NAOENCONTRADO.csv' COM UM TOTAL DE 1583 REGISTROS QUE NÃO ESTÃO SENDO CARREGADOS POR CAUSA DO PREENCHIMENTO DO CAMPO ELEMENTOPEP

### UM NOVO RELACIONAMENTO É FEITO ENTRE OS DATAFRAMES "result_GERAL_NE" E "resultdflancamento_contabil0" PARA RETORNAR OS 23 CASOS NÃO ENCONTRADOS



In [69]:

final = pd.merge(left=result_GERAL_NE
                  , right=resultdflancamento_contabil0
                  , left_on='CodDocVenda_NaoEncontrado'
                  , right_on='CodDocVenda'  
                  , how='left'
)[['CodDocVenda_NaoEncontrado_x','CodDocVenda_y','CodDocCompra']]

final.loc[final['CodDocVenda_NaoEncontrado_x']== final['CodDocVenda_y'],'Comparacao'] = 'Existe'
final.loc[final['CodDocVenda_NaoEncontrado_x']!= final['CodDocVenda_y'],'Comparacao'] = 'Não Existe'

#final.groupby('Comparacao').agg(Qtd=('Comparacao','count'))

restante = final[final['Comparacao']=='Não Existe'].sort_values(by='CodDocVenda_NaoEncontrado_x')

restante.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 52 to 1601
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   CodDocVenda_NaoEncontrado_x  23 non-null     object
 1   CodDocVenda_y                0 non-null      object
 2   CodDocCompra                 0 non-null      object
 3   Comparacao                   23 non-null     object
dtypes: object(4)
memory usage: 920.0+ bytes


# NO ARQUIVO 'C:\Temp\Frete_Rodrigo\documentos_27.csv' SÃO ARMAZENADOS O RELACIONAMENTO ENTRE ORDEM DE VENDA E DOCUMENTO CONTÁBIL PARA PORDER TRAÇAR UM COMPARATIVO ENTRE OS DADOS QUE ESTÃO SENDO CARREGADOS

In [72]:
restante = final[final['Comparacao']=='Não Existe'].sort_values(by='CodDocVenda_NaoEncontrado_x')

arquivo_documentos_27 = 'C:\\Temp\\Frete_Rodrigo\\documentos_27.csv'

dfdocumentos_27 = pd.read_csv(arquivo_documentos_27
                              , sep=','
                              , names=['CodDocVenda','CodDocCompra']
                              , dtype={'CodDocVenda':'str'
                                       ,'CodDocCompra':'str'}
                             ,header=0)

restante2 = pd.merge(left=restante
                  , right=dfdocumentos_27
                  , left_on='CodDocVenda_NaoEncontrado_x'
                  , right_on='CodDocVenda'  
                  ,how='left'
)[['CodDocVenda_NaoEncontrado_x','CodDocCompra_y']]


restante2

,CodDocVenda_NaoEncontrado_x,CodDocCompra_y
0,4001725424,4501606358
1,4001732983,4501593544
2,4001732983,4501593424
3,4001732983,4501593567
4,4001734635,4501592292
5,4001737213,4501592291
6,4001737213,4501592291
7,4001765398,4501606362
8,4001765398,4501606362
9,4001774417,4501636195


# APÓS A EXTRAÇÃO UMA NOVA COMPARAÇÃO COM OS DADOS DA ODS (ONPREMISE) É ESTABELECIDO PARA VALIDAR OS CAMPOS ORDEM INTERNA (CODORDEMINTERNA), CONTA CONTÁBIL (CODCONTACONTABIL) E ELEMENTO PEP

In [70]:
arquivo_lancamento_contabil_ods_parte02 = 'C:\\Temp\\Frete_Rodrigo\\lancamento_contabil_ods_parte02.csv'

dflancamento_contabil_ods_parte02 = pd.read_csv(arquivo_lancamento_contabil_ods_parte02
                                    , sep=';'
                                    ,names=['NumDocCompra'
                                            ,'CodTipoRegistro'
                                            ,'CodLedger'
                                            ,'NomOperacaoRef'
                                            ,'Id_Ordem'
                                            ,'CodOrdemInterna'
                                            ,'Id_ElementoPEP'
                                            ,'Id_ContaContabil'
                                            ,'CodContaContabil'
                                            ,'Id_CentroCusto'
                                            ,'Id_CentroLucro']
                                    ,dtype={'NumDocCompra':'str'
                                            ,'CodTipoRegistro':'str'
                                            ,'CodLedger':'str'
                                            ,'NomOperacaoRef':'str'
                                            ,'Id_Ordem':'str'
                                            ,'CodOrdemInterna':'str'
                                            ,'Id_ElementoPEP':'str'
                                            ,'Id_ContaContabil':'str'
                                            ,'CodContaContabil':'str'
                                            ,'Id_CentroCusto':'str'
                                            ,'Id_CentroLucro':'str'}
                                   ,header=0)


dflancamento_contabil_ods_parte02

#dflancamento_contabil['CodDocCompra'].drop_duplicates().to_csv('C:\\Temp\\Frete_Rodrigo\\documentos.csv', index=False)

,NumDocCompra,CodTipoRegistro,CodLedger,NomOperacaoRef,Id_Ordem,CodOrdemInterna,Id_ElementoPEP,Id_ContaContabil,CodContaContabil,Id_CentroCusto,Id_CentroLucro
0,4501712656,0,2L,RMRP,141,000200000140,NaN,760,0000532003,NaN,21
1,4501990774,0,2L,RMRP,81,000200000080,NaN,760,0000532003,NaN,9
2,4501788536,0,2L,RMRP,236,000200000235,NaN,760,0000532003,NaN,48
3,4501742036,0,2L,RMRP,141,000200000140,NaN,760,0000532003,NaN,21
4,4501933669,0,2L,RMRP,81,000200000080,NaN,760,0000532003,NaN,9
5,4501933668,0,2L,RMRP,81,000200000080,NaN,760,0000532003,NaN,9
6,4501764797,0,2L,RMRP,236,000200000235,NaN,760,0000532003,NaN,48
7,4501907459,0,2L,RMRP,81,000200000080,NaN,760,0000532003,NaN,9
8,4501964683,0,2L,RMRP,81,000200000080,NaN,760,0000532003,NaN,9
9,4501730219,0,2L,RMRP,141,000200000140,NaN,760,0000532003,NaN,21


In [105]:
final_restante = pd.merge(left=restante2
                  , right=dflancamento_contabil_ods_parte02
                  , left_on='CodDocCompra_y'
                  , right_on='NumDocCompra'  
                  , how='left'
)[['CodDocVenda_NaoEncontrado_x','NumDocCompra','CodOrdemInterna','CodContaContabil','Id_ElementoPEP']]

final_restante.fillna('-1',inplace=True)

final_restante[final_restante['NumDocCompra']!='-1'].drop_duplicates()

final_restante = final_restante[final_restante['NumDocCompra']!='-1'].drop_duplicates(keep = 'last').reset_index(drop = True)

final_restante.loc[~final_restante['CodContaContabil'].isin(['0000532001'
                                                            ,'0000532002'
                                                            ,'0000532003'
                                                            ,'0000532004']) ,'Comparacao'] = 'Conta Não Mapeada'

final_restante.loc[final_restante['CodOrdemInterna']!='-1' ,'Comparacao'] = 'Ordem Interna Preenchida'

final_restante.loc[final_restante['Id_ElementoPEP']!='-1' ,'Comparacao'] = 'ElementoPEP Preenchido'

final_restante[final_restante['CodContaContabil']=='0000536001'].sort_values(by='Comparacao').to_csv('C:\\Temp\\Frete_Rodrigo\\comparacao_GERAL_NAOENCONTRADO_CONTACONTABIL_0000536001.csv', index=False
                                   , encoding='latin-1'
                                   , sep=';')


final_restante[final_restante['CodOrdemInterna']!='-1'].sort_values(by='Comparacao').to_csv('C:\\Temp\\Frete_Rodrigo\\comparacao_GERAL_NAOENCONTRADO_ORDEMINTERNA_PREENCHIDA.csv', index=False
                                   , encoding='latin-1'
                                   , sep=';')